In [47]:
%cd ../
%pwd

D:\Projects\Cx


'D:\\Projects\\Cx'

In [48]:
from sims_pars import bayes_net_from_script
from sims_pars.fitting import AbsObjectiveSimBased
import scipy.stats as sts

In [49]:
class Test(AbsObjectiveSimBased):
    def __init__(self):
        bn = bayes_net_from_script('''
        PCore Normal2 {
            r_die_tb_untx ~ unif(0.07, 0.19)
            rr_die_asym ~ unif(0, 1)

            prev_rural ~ unif(0.001, 0.01)
            prev_urban ~ unif(0.001, 0.01)
            prev_slum ~ unif(0.001, 0.01)

            // Life history
            r_sc ~ unif(0.15, 0.25)  # self-cure

            // Care-seeking and diagnosis
            rr_det_urban ~ unif(0.5, 5)
            rr_det_slum ~ unif(0.5, 5)

            p_dx_pri_frac ~ unif(0.5, 1)
            p_dx_pub = 0.83
            p_dx_eng = (p_dx_pub + p_dx_pri) / 2
            p_dx_pri = p_dx_pub * p_dx_pri_frac

            p_txi_pub = 0.88
            p_txi_eng = 0.88
            p_txi_pri = 0.75

            p_public ~ unif(0.3, 0.8)
            p_eng_rural_frac ~ unif(0, 1)
            p_eng_rural = p_eng_urban * p_eng_rural_frac
            p_eng_urban ~ unif(0, 1)
            p_eng_slum = p_eng_urban
            r_acf ~ unif(0, 0.05)

            ppv = 0.85

            // Treatment
            r_tx_succ = 2
            dur_succ_ds = 0.5
            dur_succ_dr = 2
            x ~ binom(size=60, prob = 0.5)
        }
        ''')
        AbsObjectiveSimBased.__init__(self, bn)

    def simulate(self, pars):
        return 'test'

    def link_likelihood(self, sim):
        return 1

In [50]:
obj = Test()

In [52]:
p0 = obj.sample_prior()

In [57]:
for k, v in dict(p0).items():
    print(k, 'int' if isinstance(v, int) else 'float')

r_die_tb_untx float
rr_die_asym float
prev_rural float
prev_urban float
prev_slum float
r_sc float
rr_det_urban float
rr_det_slum float
p_dx_pri_frac float
p_dx_pub float
p_txi_pub float
p_txi_eng float
p_txi_pri float
p_public float
p_eng_rural_frac float
p_eng_urban float
r_acf float
ppv float
r_tx_succ int
dur_succ_ds float
dur_succ_dr int
x int
p_dx_pri float
p_eng_rural float
p_eng_slum float
p_dx_eng float


In [38]:
p = obj.sample_prior()
obj.evaluate(p)
p.to_json()

{'Locus': {'r_die_tb_untx': 0.15626521599792703,
  'rr_die_asym': 0.6795213509770783,
  'prev_rural': 0.008548356397980902,
  'prev_urban': 0.005254825478947948,
  'prev_slum': 0.003550808921758208,
  'r_sc': 0.15283906681589926,
  'rr_det_urban': 0.6340713542661383,
  'rr_det_slum': 4.198613710488985,
  'p_dx_pri_frac': 0.8329800169577917,
  'p_dx_pub': 0.83,
  'p_txi_pub': 0.88,
  'p_txi_eng': 0.88,
  'p_txi_pri': 0.75,
  'p_public': 0.4628655675576188,
  'p_eng_rural_frac': 0.4219831697206283,
  'p_eng_urban': 0.43679207877987447,
  'r_acf': 0.041486259029210756,
  'ppv': 0.85,
  'r_tx_succ': 2,
  'dur_succ_ds': 0.5,
  'dur_succ_dr': 2,
  'p_dx_pri': 0.6913734140749671,
  'p_eng_rural': 0.18431890591239383,
  'p_eng_slum': 0.43679207877987447,
  'p_dx_eng': 0.7606867070374835},
 'LogPrior': 19.928312575253223,
 'LogLikelihood': 1}

In [39]:
obj.FreeParameters

['r_die_tb_untx',
 'rr_die_asym',
 'prev_rural',
 'prev_urban',
 'prev_slum',
 'r_sc',
 'rr_det_urban',
 'rr_det_slum',
 'p_dx_pri_frac',
 'p_public',
 'p_eng_rural_frac',
 'p_eng_urban',
 'r_acf']

In [41]:
step_sizes = {
    'prev_rural': 0.001,
    'prev_urban': 0.001,
    'prev_slum': 0.001,
    'r_sc': 0.05,
    'rr_det_urban': 0.1,
    'rr_det_slum': 0.1,
    'p_dx_pri_frac': 0.1,
    'p_public': 0.1,
    'p_eng_rural_frac': 0.5
}

def mutate1(p0, sizes):
    p = p0.clone()
    changes = {k: v + sizes[k] for k, v in p.Locus.items() if k in sizes}

    p.impulse(changes)
    obj.evaluate_prior(p)
    return p

%timeit mutate1(p, step_sizes)

8.11 ms ± 234 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [42]:
def mutate2(p0, sizes):
    changes = {k: v + sizes[k] for k, v in p0.Locus.items() if k in sizes}

    return obj.serve(changes)

%timeit mutate2(p, step_sizes)

10.2 ms ± 356 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


3